## Saving multiple files at once - pygix - transform() & orientation()
Pygix is a generic python library for performing reduction of grazing-incidence and fiber X-ray scattering data.

The pygix library has been developed to reduce 2D X-ray scattering data for experiments 
recorded in grazing-incidence (GISAXS, GIWAXS, GIXRD, GIXD, collectively referred to as GIXS) and fiber diffraction modes.
Both 2D image projections and 1D line profile extraction are handled. The package is designed to be as generic as possible,
i.e., it makes no assumptions about the data (passed by the user as numpy arrays) and all detector and geometric corrections
can be handled at the point of data reduction.

In [6]:
#Import all of the needed packages

import numpy as np #package that contains a large collection of mathematical functions
import pylab #module in Matplotlib - bulk imports matplotlib
import math 
import matplotlib.pyplot as plt #collection of functions that make matplotlib work like MATLAB
import matplotlib.gridspec as gridspec #grid layout to place subplots within a figure
import scipy.optimize #provides functions to minimize or maximize objective functions
from scipy.optimize import curve_fit #function meant to calculate a model for some phenomenon - finds best match
import pyFAI #Fast Azimunthal Integration in Python
import fabio #used for reading of raw 2D data from various X-ray detectors
import pandas as pd #library used for data manipulation and analysis
from xrdfunctions import gaussian, lorentz, pvoigt #import the functions that are methods used to analyze data from spectroscopy or diffraction
                                                #these functions are available and have been created within the another .py file. In order to use
                                                #these functions, that xrdfunctions.py file must be in the same folder as this file.
import glob #A string of characters used to match file paths
import pygix

# C60 DATASET

In [7]:
#Obtain the path of the .tif files

#path where .tif files are located, the files in the folder will be read, hence the 'r'

## Make sure to mark the path to where the .tif files are located
## If this python file is located in the same folder in which the .tif data is located, there is no need to mark a path.
path =  r'C:/Users/kimhd/Hernandez_Kim_Solar_Lab/D6a_MAPbIBr2_C60_lightcycle-20210506T021405Z-001/D6a_MAPbIBr2_C60_lightcycle'
all_files = sorted(glob.glob(path+"/*.tif")) #all of the .tif files will be obtains and sorted in ascending order

Since this experiment was taken at alternating angles, .4 to 3 degrees, those specific angles correpond to the an even or odd .tif file collected at the site.

- even .tif file = surface data = incident deg is .4
- odd .tif tile = bulk data = incident deg is 3

In [8]:
#Since this experiment was taken at alternating angles, .4 to 3 degrees, those specific angles correspond to the an even or odd .tif file
#collected at the site. 
# even .tif file = surface data = .4 deg
# odd .tif tile = bulk data = 3 deg

#The code below alternates throughout those files, saves the corrected file as a png and another png that contains the
#plot intensity versus scattering angle of the first peak

#The code below will take a while, roughly ~40 minutes

image_num=0; #Variable used as a counter, to loop throughout all of the images

while image_num < len(all_files):
    
    if image_num%2 == 0:
        ##ODD
        
        #Load the calibration file (.poni) and set the geometry
        pg = pygix.Transform() #obtain the Transform class pygix
        #When it comes to loading the .poni file, make sure that insert the path of where the file can be found
        #or you can use type in the .poni file by itself if the .poni file is within the same folder of this .py file.
        pg.load('C:/Users/kimhd/Hernandez_Kim_Solar_Lab/LaB6_det315_3s_01221009_0001.poni') #load the calibration file
        #Loading the .poni file note:
            #The line above works if the .poni file is in the same folder
            #as this .py file. Other wise use the full path to the .poni file
        pg.incident_angle=.4 #Incident angle used, 3 for bulk data (odd) #.4 for surface (even)
        pg.sample_orientation = 3    # set sample orientation: 1 is horizontal, 
        #2 is vertical, 3 is horizontal rotated by 180 deg, 4 is vertical rotated by 180 deg
        pg.tilt_angle = 0 #[deg] tilt angle of sample
        pg
        
        
        
        #Open the .tif image and save it as "img"
        img = fabio.open(all_files[image_num]).data
    
        ## TRANSFORMATION ##
    
        # Run calibration, accounting for solid angle correction using .transform_reciprocal
        ii_2d, qxy_2d, qz_2d = pg.transform_reciprocal(img, correctSolidAngle=True, method='bbox', npt=3000)
        #returns the intensity, qxy, and qz - transforms raw data to anguler or q coordinates
        fig2, ax2 = plt.subplots()
        ax2.set_xlabel("q$_\mathregular{xy}$ ($\mathring{A}^\mathregular{-1}$)")
        ax2.set_ylabel("q$_\mathregular{z}$ ($\mathring{A}^\mathregular{-1}$)") 
        ax2.yaxis.set_ticks_position('both')
        ax2.xaxis.set_ticks_position('both')
        ax2.imshow(ii_2d, extent=(np.min(qxy_2d)/10,np.max(qxy_2d)/10,np.min(qz_2d)/10,np.max(qz_2d)/10), vmin = 3, vmax = 100, origin='lower') 
        plt.ylim(0,3.5)
        plt.xlim(-2,2)
    
        ##NOTE: Below I have stated the main tranform methods this package provides
        #I, qxy, qz = pg.transform_reciprocal(data)
        #Transform raw image to angular or q coordinates
        #I, q, chi  = pg.transform_polar(data)
        #Transform image to cake plot (chi vs q or 2theta)
        #I, q = gi.integrate_1d(data, npt)
            #1d integration

    
        #Save the corrected file into a png
        #type path
        
        #Make a path of folder in which you could like your images to be saved to
        path = "C:/Users/kimhd/Hernandez_Kim_Solar_Lab/D6a_MAPbIBr2_C60_lightcycle-20210422T232211Z-001/D6a_MAPbIBr2_C60_lightcycle/"
        #obtain only the .tif file name 
        tif_file = all_files[image_num][-44:-4] + '.tif'
    
        temp = tif_file.split(sep='.')
        #label the corrected png image
        filename = temp[0]+'_corrected.png'
        #Save the png image to your desired path
        fig2.savefig(path+filename, bbox_inches='tight', dpi=900, transparent=True) #save the png image into the path listed
        #clear the figure created in order to save memory, without clearing the figures, this cell will be extremely slow
        #and not work
        plt.cla()
        plt.clf()
        plt.close()
    
    
        ## ORIENTATION ##
        #Since our material is cubic, we will be okay by just investigating the first peak
        #since all of the other peaks will shift in orientation just the same as the first peak.
    
        # 1) FIND THE PEAK
        #profile_sector will be used to help us look for the first peak between 1 and 1.1 Ang^-1 aka. radial_range = (9,11)
        Itest, q = pg.profile_sector(img, correctSolidAngle=True, radial_range = (9, 11), method='bbox', npt=3000) 
        p0 = [.1, 140, 10.4, .2] #initial guess at fit paramenters
        #Conduct a fit peak using psuedovoigt, returns 
        # popt = Optimal parameters
        # pcov = covariance matrix based on scaling sigma by a constant factor
        popt,pcov = curve_fit(pvoigt, q, Itest-Itest[0], p0, maxfev=6000)
        #Obtain the center and width
        center = popt[2]
        width = popt[3]

        # 2) 
        #Using parameters from 1D to plot intensity versus scattering angle of the discovered peak
        back, chi = I, chi = pg.profile_chi(img, correctSolidAngle=True, radial_pos = center+1, radial_width = 2*width, chi_range = (-90,90), method='bbox', npt=3000) #looks at range from 0 to 90 degrees
        I, chi = pg.profile_chi(img, correctSolidAngle=True, radial_pos = center, radial_width = 2*width, chi_range = (0,90), method='bbox', npt=3000)
        Iback = I-back #correct for background intensity
        Icorrect = Iback*np.sin(chi*math.pi/180) #correct for amount of scattered material
        #Create figures
        fig3, ax3 = plt.subplots()
        ax3.set_xlabel('$\chi$ ($^\circ$)')
        ax3.set_ylabel('Intensity (a.u.)')
        plt.plot(chi,Icorrect)

    
        #save the orientation figures
        temp = tif_file.split(sep='.')
        filename = temp[0]+'_orientation.png'
        fig3.savefig(path+filename, bbox_inches='tight', dpi=900, transparent=True)
        #clear the figure created in order to save memory, without clearing the figures, this cell will be extremely slow
        #and not work
        plt.cla()
        plt.clf()
        plt.close()
        
        
        
        
        
        
        
        
    elif image_num%2 ==1:
        ##EVEN
        
        #Load the calibration file (.poni) and set the geometry
        pg = pygix.Transform() #obtain the Transform class pygix
        
        #When it comes to loading the .poni file, make sure that insert the path of where the file can be found
        #or you can use type in the .poni file by itself if the .poni file is within the same folder of this .py file.
        pg.load('C:/Users/kimhd/Hernandez_Kim_Solar_Lab/LaB6_det315_3s_01221009_0001.poni') #load the calibration file
        #Loading the .poni file note:
            #The line above works if the .poni file is in the same folder
            #as this .py file. Other wise use the full path to the .poni file
        pg.incident_angle=.4 #Incident angle used, 3 for bulk data (odd) #.4 for surface (even)
        pg.sample_orientation = 3    # set sample orientation: 1 is horizontal, 
        #2 is vertical, 3 is horizontal rotated by 180 deg, 4 is vertical rotated by 180 deg
        pg.tilt_angle = 0 #[deg] tilt angle of sample
        pg
        
        
        #Open the .tif image and save it as "img"
        img = fabio.open(all_files[image_num]).data
    
        ## TRANSFORMATION ##
    
        # Run calibration, accounting for solid angle correction using .transform_reciprocal
        ii_2d, qxy_2d, qz_2d = pg.transform_reciprocal(img, correctSolidAngle=True, method='bbox', npt=3000)
        #returns the intensity, qxy, and qz - transforms raw data to anguler or q coordinates
        #create figure and axis
        fig2, ax2 = plt.subplots()
        ax2.set_xlabel("q$_\mathregular{xy}$ ($\mathring{A}^\mathregular{-1}$)")
        ax2.set_ylabel("q$_\mathregular{z}$ ($\mathring{A}^\mathregular{-1}$)") 
        ax2.yaxis.set_ticks_position('both')
        ax2.xaxis.set_ticks_position('both')
        ax2.imshow(ii_2d, extent=(np.min(qxy_2d)/10,np.max(qxy_2d)/10,np.min(qz_2d)/10,np.max(qz_2d)/10), vmin = 3, vmax = 100, origin='lower') 
        plt.ylim(0,3.5)
        plt.xlim(-2,2)

        ##NOTE: Below I have stated the main tranform methods this package provides
        #I, qxy, qz = pg.transform_reciprocal(data)
        #Transform raw image to angular or q coordinates
        #I, q, chi  = pg.transform_polar(data)
        #Transform image to cake plot (chi vs q or 2theta)
        #I, q = gi.integrate_1d(data, npt)
        #1d integration

    
        #Save the corrected file into a png
        #type path
        #Make a path of folder in which you could like your images to be saved to
        path = "C:/Users/kimhd/Hernandez_Kim_Solar_Lab/D6a_MAPbIBr2_C60_lightcycle-20210422T232211Z-001/D6a_MAPbIBr2_C60_lightcycle/"
        #obtain only the .tif file name 
        tif_file = all_files[image_num][-44:-4] + '.tif'
    
        temp = tif_file.split(sep='.')
        #label the corrected png image
        filename = temp[0]+'_corrected.png'
        #Save the png image to your desired path
        fig2.savefig(path+filename, bbox_inches='tight', dpi=900, transparent=True) #save the png image into the path listed
    
        #clear the figure created in order to save memory, without clearing the figures, this cell will be extremely slow
        #and not work
        plt.cla()
        plt.clf()
        plt.close()
    
    
    
        ## ORIENTATION ##
        #Since our material is cubic, we will be okay by just investigating the first peak
        #since all of the other peaks will shift in orientation just the same as the first peak.
    
        # 1) FIND THE PEAK
        #profile_sector will be used to help us look for the first peak between 1 and 1.1 Ang^-1 aka. radial_range = (10,11)
        Itest, q = pg.profile_sector(img, correctSolidAngle=True, radial_range = (10, 11), method='bbox', npt=3000) 
        p0 = [.1, 140, 10.4, .2] #initial guess at fit paramenters
        #Conduct a fit peak using psuedovoigt, returns 
        # popt = Optimal parameters
        # pcov = covariance matrix based on scaling sigma by a constant factor
        popt,pcov = curve_fit(pvoigt, q, Itest-Itest[0], p0, maxfev=6000)
        #Obtain the center and width
        center = popt[2]
        width = popt[3]

        # 2) 
        #Using parameters from 1D to plot intensity versus scattering angle of the discovered peak
        back, chi = I, chi = pg.profile_chi(img, correctSolidAngle=True, radial_pos = center+1, radial_width = 2*width, chi_range = (0,90), method='bbox', npt=3000) #looks at range from 0 to 90 degrees
        I, chi = pg.profile_chi(img, correctSolidAngle=True, radial_pos = center, radial_width = 2*width, chi_range = (0,90), method='bbox', npt=3000)
        Iback = I-back #correct for background intensity
        Icorrect = Iback*np.sin(chi*math.pi/180) #correct for amount of scattered material
        #create figures
        fig3, ax3 = plt.subplots()
        ax3.set_xlabel('$\chi$ ($^\circ$)')
        ax3.set_ylabel('Intensity (a.u.)')
        plt.plot(chi,Icorrect)

    
        #save the orientation figures
        temp = tif_file.split(sep='.')
        filename = temp[0]+'_orientation.png'
        fig3.savefig(path+filename, bbox_inches='tight', dpi=900, transparent=True)
        #clear the figure in order to save memory
        plt.cla()
        plt.clf()
        plt.close()
        
        
    
    image_num = image_num+1
